In [31]:
%pip install "git+https://github.com/ScierKnave/TorchMPS.git"
!git clone "https://github.com/ScierKnave/honor_project.git"

import numpy as np
import torch
import torch.nn.functional as F
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
from torch.nn.modules.pooling import MaxPool2d
%pip install torchmetrics
from torchmetrics.classification import MulticlassAccuracy
from torchmps import MPS
import math

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/ScierKnave/TorchMPS.git to /tmp/pip-req-build-oebwzbvh
  Running command git clone --filter=blob:none --quiet https://github.com/ScierKnave/TorchMPS.git /tmp/pip-req-build-oebwzbvh
  Resolved https://github.com/ScierKnave/TorchMPS.git to commit f716a08e15d0af50dbfdfc435ab9604e82562ea3
  Preparing metadata (setup.py) ... done
fatal: destination path 'honor_project' already exists and is not an empty directory.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


# Hyperparameters

In [32]:
# FC to MPS

# Hardware hyperparameters
chosen_device = torch.device('cuda' 
if torch.cuda.is_available() else 'cpu')

# Data hyperparameters
nb_train_HP = 2000
nb_test_HP = 500
batch_sz_HP = 150
batch_sz_HP = min(batch_sz_HP, nb_train_HP)
nb_classes_HP = 10

# Teacher hyperparameters
nepochs_teacher_HP = 25
teacher_loss_HP = nn.CrossEntropyLoss()
teacher_lr_HP = 1e-2
teacher_reg_HP = 0.01
teacher_hidden_size_HP = 70
# Student hyperparameters
# MPS parameters
bond_dim_HP = 10
adaptive_mode_HP = False
periodic_bc_HP = False

# USING CUSTOM FEATURE MAP WITH FORK?: YES

# Training parameters
nepochs_student_HP = 25 
student_lr_HP = 1e-4
student_reg_HP = 0.01
student_loss_HP = nn.KLDivLoss(reduction = "batchmean", log_target = True)

# Gaussian parameters
gauss_epochs_HP = 25 # number of epochs with added gaussian noise
gn_var_HP = 0.2 #added gaussian noise variance
gn_mean_HP = 0 #added gaussian noise mean
nepochs_student_HP = nepochs_student_HP + gauss_epochs_HP


# Premilinaries: Importing the data and utils subroutines

In [33]:
# Import the *common* mnist train set and create a batch iterator for it.
train_set = torch.load('/content/honor_project/src/datasets/train_mnist.pt')
train_iterator = torch.utils.data.DataLoader(
    dataset = train_set, 
    sampler = torch.utils.data.SubsetRandomSampler(range(nb_train_HP)),
    batch_size=batch_sz_HP
    )

# Import the mnist *common* test set and create a batch iterator for it.
test_set = torch.load('/content/honor_project/src/datasets/test_mnist.pt')
test_iterator = torch.utils.data.DataLoader(
    dataset = test_set, 
    sampler = torch.utils.data.SubsetRandomSampler(range(nb_test_HP)),
    batch_size = batch_sz_HP
    )

In [34]:
# Returns the validation set classification accuracy
# of the given input model (this is a higher order function)
def get_acc(model, iterator, reshape, dataset_size):
    # Get the validation set classification accuracy
    total_good_classifications = 0
    acc_metric = MulticlassAccuracy(num_classes=nb_classes_HP).to(chosen_device)
    for (x_mb, y_mb) in iterator:
        if reshape == True: 
          x_mb = x_mb.reshape(-1, 784)
        x_mb = x_mb.to(chosen_device)
        y_mb = y_mb.to(chosen_device)
        # Add the number of datapoints we classified right to the total
        batch_size = x_mb.size()[0]
        y_hat = model(x_mb)
        batch_good_classifications = batch_size * acc_metric(y_hat, y_mb)
        total_good_classifications += batch_good_classifications
    return total_good_classifications / dataset_size # divide by total size

In [35]:
# Create the fcnn class
import torch.nn as nn
class FCNN(nn.Module):
    def __init__(self):
        super(FCNN, self).__init__()
        self.conv1 = nn.Sequential(
            nn.Conv2d(1, 128, 5),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )
        self.conv2 = nn.Sequential(
            nn.Conv2d(128, 64, 3),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )
        self.lin1= nn.Linear(1600, 10)
    def forward(self, x):
      x = self.conv1(x)
      x = self.conv2(x)
      x = torch.flatten(x, start_dim=1)
      x = self.lin1(x)
      return x

teacher = torch.load('/content/honor_project/src/cnn_0.977.pt')


# Training the student model

In [36]:
def train_a_student():
  '''
  Trains a student model.
  '''
  # Initialize the MPS module
  student = MPS(
      input_dim = 28 ** 2,
      output_dim = 10,
      bond_dim = bond_dim_HP
  ).to(chosen_device)
  #student.register_feature_map(feature_map_HP)

  
  model_parameters = filter(lambda p: p.requires_grad, student.parameters())
  params = sum([np.prod(p.size()) for p in model_parameters])
  print(params)

  # Instantiate the optimizer and softmax
  student_optimizer = torch.optim.Adam(
      student.parameters(), lr = student_lr_HP, weight_decay = student_reg_HP
  )

  # Used on the inputs before the loss function
  LogSoftmax = nn.LogSoftmax(dim=1)

  # Create an array to store the val loss
  # of the student at each epoch
  stud_test_loss = np.array([])
  stud_train_loss = np.array([])

  # Training loop 
  for epoch in range(nepochs_student_HP):
      for (x_mb, y_mb) in train_iterator:
          # Flatten the MNIST images, which come in matrix form
          x_mb = x_mb.to(chosen_device)
          y_mb = y_mb.to(chosen_device)

          # Add Gaussian noise for the gaussian epochs
          if (epoch <= (nepochs_student_HP - gauss_epochs_HP)):
            x_mb = x_mb + torch.randn(size=x_mb.size()).to(chosen_device)

          student_output = LogSoftmax( student(x_mb.reshape(-1, 784)) )
          teacher_output = LogSoftmax( teacher(x_mb) )

          # Backpropagation
          loss = student_loss_HP(student_output, teacher_output)
          loss.backward()
          student_optimizer.step()
          student_optimizer.zero_grad()

      # Get accuracy over all test and training data for current epoch
      train_current_accuracy = round( get_acc(student, train_iterator, True, nb_train_HP).item(), 4)
      test_current_accuracy = round( get_acc(student, test_iterator, True, nb_test_HP).item(), 4)
      stud_train_loss = np.append(stud_train_loss, train_current_accuracy)
      stud_test_loss = np.append(stud_test_loss, test_current_accuracy)
      print(stud_test_loss)
  return(stud_train_loss, stud_test_loss)

# Repeat the training process in order to get the variance
global_stud_test_loss = np.array([])
global_stud_train_loss = np.array([])
for i in range(20):
  print("\n Training and testing of model ", i+1, " in progress...")
  (stud_train_loss, stud_test_loss) = train_a_student()
  print("Train loss: ", stud_train_loss)
  print("Test loss: ", stud_test_loss)
  if (i == 0):
    global_stud_train_loss = stud_train_loss
    global_stud_test_loss = stud_test_loss
  else:
    global_stud_train_loss = np.vstack((global_stud_train_loss, stud_train_loss))
    global_stud_test_loss = np.vstack((global_stud_test_loss, stud_test_loss))



 Training and testing of model  1  in progress...
158000
[0.1]
[0.1 0.1]
[0.1 0.1 0.1]
[0.1 0.1 0.1 0.1]
[0.1 0.1 0.1 0.1 0.1]
[0.1 0.1 0.1 0.1 0.1 0.1]
[0.1 0.1 0.1 0.1 0.1 0.1 0.1]
[0.1    0.1    0.1    0.1    0.1    0.1    0.1    0.1075]
[0.1    0.1    0.1    0.1    0.1    0.1    0.1    0.1075 0.1667]
[0.1    0.1    0.1    0.1    0.1    0.1    0.1    0.1075 0.1667 0.227 ]
[0.1    0.1    0.1    0.1    0.1    0.1    0.1    0.1075 0.1667 0.227
 0.2705]
[0.1    0.1    0.1    0.1    0.1    0.1    0.1    0.1075 0.1667 0.227
 0.2705 0.3171]
[0.1    0.1    0.1    0.1    0.1    0.1    0.1    0.1075 0.1667 0.227
 0.2705 0.3171 0.334 ]
[0.1    0.1    0.1    0.1    0.1    0.1    0.1    0.1075 0.1667 0.227
 0.2705 0.3171 0.334  0.3315]
[0.1    0.1    0.1    0.1    0.1    0.1    0.1    0.1075 0.1667 0.227
 0.2705 0.3171 0.334  0.3315 0.3351]
[0.1    0.1    0.1    0.1    0.1    0.1    0.1    0.1075 0.1667 0.227
 0.2705 0.3171 0.334  0.3315 0.3351 0.3739]
[0.1    0.1    0.1    0.1    0.1    0.1   

In [39]:
# Print the final results and save them for the report.
print("Final results")
print("Train loss: ", global_stud_train_loss)
print("Test loss: ", global_stud_test_loss)

np.save('20x_cnn_to_mps_trainloss' + str(bond_dim_HP) + 'bd', global_stud_train_loss)
np.save('20x_cnn_to_mps_testloss' + str(bond_dim_HP) + 'bd', global_stud_test_loss)



Final results
Train loss:  [[0.1    0.1    0.1    0.1    0.1    0.1    0.1    0.1114 0.1551 0.2138
  0.2593 0.2934 0.3183 0.3289 0.3403 0.3634 0.3686 0.3989 0.4346 0.3452
  0.3821 0.4094 0.412  0.4696 0.4353 0.4776 0.7649 0.7815 0.8735 0.886
  0.8912 0.9151 0.9192 0.9295 0.9216 0.9263 0.9053 0.9412 0.9533 0.954
  0.9582 0.9538 0.949  0.9487 0.9549 0.9596 0.9581 0.9701 0.968  0.9765]
 [0.1    0.1    0.1    0.1    0.1    0.1    0.1    0.1195 0.1768 0.1768
  0.2393 0.2979 0.3256 0.3656 0.3531 0.3569 0.3712 0.3773 0.3765 0.3949
  0.3633 0.4008 0.4021 0.4451 0.4285 0.4765 0.7242 0.7451 0.8067 0.873
  0.883  0.9094 0.9244 0.9136 0.9367 0.9433 0.9491 0.9459 0.9539 0.9539
  0.9625 0.9597 0.9696 0.964  0.973  0.9707 0.9636 0.971  0.9795 0.9719]
 [0.1    0.1    0.1    0.1    0.1    0.1    0.177  0.1845 0.212  0.2597
  0.2859 0.3021 0.3008 0.3593 0.3802 0.3318 0.3625 0.3569 0.4081 0.4231
  0.4472 0.401  0.4769 0.4119 0.4967 0.4392 0.736  0.7958 0.8589 0.8626
  0.8864 0.9086 0.9134 0.9325 0.9131 0